# Colorization of Densified Point Clouds

The densified point cloud ignores all color in the original point cloud. This script uses the original point cloud to colorize the densified point cloud based on a weighted nearest neighbor search.

In [1]:
# Original point cloud is given in .npy format, with XYZRGBL data
# Densified point cloud is given in .xyz format, with XYZ data (can be read as plain text)

import numpy as np
import glob
from sklearn.neighbors import KDTree
import tqdm

original_point_cloud_paths =  glob.glob("buildings_split/*.npy")
densified_point_cloud_paths = glob.glob("buildings_split/*.xyz")

def write_ply(filename, data):
    with open(filename, 'w') as f:
        f.write("ply\n")
        f.write("format ascii 1.0\n")
        f.write("element vertex {}\n".format(len(data)))
        f.write("property float x\n")
        f.write("property float y\n")
        f.write("property float z\n")
        f.write("property uchar red\n")
        f.write("property uchar green\n")
        f.write("property uchar blue\n")
        f.write("end_header\n")
        for row in data:
            f.write("{} {} {} {} {} {}\n".format(row[0], row[1], row[2], int(row[3]), int(row[4]), int(row[5])))

# If a point in the densified point cloud is the same as a point in the original point cloud, it is given the color of the original point cloud
# we will need to take that into account when weighting the nearest neighbors

for i in tqdm.tqdm(range(len(densified_point_cloud_paths))):
    densified_point_cloud = np.loadtxt(densified_point_cloud_paths[i])
    # load the original point cloud with the same name as the densified point cloud
    file_name_without_extension = densified_point_cloud_paths[i].split("/")[-1].split(".")[0]
    original_point_cloud = np.load("{}.npy".format(file_name_without_extension))
    # add columns for the color of the densified point cloud and last column for the label
    densified_point_cloud = np.hstack((densified_point_cloud, np.zeros((len(densified_point_cloud), 4))))

    # build a KDTree for the original point cloud
    original_point_cloud_tree = KDTree(original_point_cloud[:, :3], leaf_size=2)
    # for each point in the densified point cloud, find the nearest neighbor in the original point cloud
    for j in range(len(densified_point_cloud)):
        # find the nearest neighbor in the original point cloud
        _, nearest_neighbor_index = original_point_cloud_tree.query(densified_point_cloud[j, :3].reshape(1, -1), k=1)
        # assign the color of the nearest neighbor to the densified point cloud
        densified_point_cloud[j, 3:6] = original_point_cloud[nearest_neighbor_index, 3:6]
        # the label is always 5 (building)
        densified_point_cloud[j, 6] = 5
        
    
    # Save the colored point cloud
    # if the file does not exist, it will be created. If it already exists, it will be overwritten
    filename = "{}_colored.ply".format(file_name_without_extension)
    write_ply(filename, densified_point_cloud)
    


 75%|███████▌  | 9/12 [19:31<06:30, 130.13s/it]

KeyboardInterrupt

